# L16a: The Curse of Dimensionality and the Rise of Deep Q Learning
In this lecture we'll revisit Q-learning (and in some sense multiplecative weights and bandit algorithms) and see how they can be used to solve problems with large (possible infinite) state/actions spaces. The key concepts that students should be able to define and demonstrate from this lecture are:

* __The curse of dimensionality__ refers to the exponential growth of the state-action space as the number of features increases, making traditional Q-learning intractable for high-dimensional problems. Deep Q-learning mitigates this by using neural networks to approximate the Q-function, enabling generalization across large or continuous state spaces.
* __Q-Learning__: Utilizes a _Q-table_ to store and update the value of each state-action pair. This approach becomes infeasible as the state-action space grows exponentially with the number of features, leading to the curse of dimensionality
* __Deep Q-Learning__: Employs a feed-foward neural network to approximate the Q-value function $Q(s,a)$, allowing for generalization across large or continuous state spaces. This method addresses the scalability issues inherent in traditional Q-learning

The sources for this lecture include:
* Fill me in here.


Let's go!!
___

## Review: Q-Learning Theory
Q-learning estimates the action-value function $Q(s, a)$ by conducting repeated experiments $t=1,2,\ldots$ in the world $\mathcal{W}$. 
In each experiment, an agent in state $s\in\mathcal{S}$ takes action $a\in\mathcal{A}$, receives a reward $r$, and (potentially) transitions to a new state $s^{\prime}$. After each experiment $t$, the agent updates its estimate of $Q(s, a)$ using the update rule:
$$
\begin{equation*}
Q_{t+1}(s,a)\leftarrow{Q_{t}(s,a)}+\alpha_{t}\cdot\underbrace{\left(r+\gamma\cdot\max_{a^{\prime}\in\mathcal{A}}Q_{t}(s^{\prime},a^{\prime}) - Q_{t}(s,a)\right)}_{\text{new information}}\quad{t = 1,2,3,\ldots}
\end{equation*}
$$
where $0<\alpha_{t} <{1}$ is the learning rate parameter at time $t$, and $0<\gamma<{1}$ is the discount factor. 
We estimate the policy function $\pi:\mathcal{S}\rightarrow\mathcal{A}$ by selecting the action $a$ that maximizes $Q(s,a)$ at each state $s$:
$$
\begin{equation*}
\pi(s) = \arg\max_{a\in\mathcal{A}}Q(s,a)
\end{equation*}
$$

### Algorithm
Initialize $Q(s,a)$ arbitrarily for all $s\in\mathcal{S}$, and $a\in\mathcal{A}$.
Set the hyperparameters: learning rate $\alpha_{t}$, the discount factor $\gamma$, the exploration rate $\epsilon_{t}$, and the convergence tolerance $\delta$.

For $s\in\mathcal{S}$
1. Initialize the time $t\gets{1}$
2. While not converged:
    1. Role a random number $p\in[0,1]$.
    2. If $p\leq\epsilon_{t}$, choose a random (uniform) action $a_{t}\in\mathcal{A}$. Otherwise, choose a greedy action $a_{t} = \text{arg}\max_{a\in\mathcal{A}}{Q_{t}(s,a)}$.
    3. Take action $a_{t}$, observe the reward $r$ from the _world_ and transition to the next state $s^{\prime}$.
    4. Update the state-action-value function: $Q_{t+1}(s,a)\leftarrow{Q_{t}(s,a)}+\alpha_{t}\cdot\underbrace{\left(r+\gamma\cdot\overbrace{\max_{a^{\prime}\in\mathcal{A}}Q_{t}(s^{\prime},a^{\prime})}^{\text{one-step lookahead}} - Q_{t}(s,a)\right)}_{\text{new information}}$.
    5. Update the state $s\leftarrow{s^{\prime}}$, the time $t\leftarrow{t+1}$, the exploration rate $\epsilon_{t+1}\leftarrow\epsilon_{t}$ and the learning rate $\alpha_{t+1}\leftarrow\alpha_{t}$.
    6. Check for convergence. if the $Q(s,a)$ has bounded change $\lVert{Q_{t+1}(s,a) - Q_{t}(s,a)}\rVert\leq\delta$, then the algorithm has _converged_. Otherwise, continue.
3. End While
4. End For

### Convergence
Q-learning converges to the optimal policy under two key theoretical conditions (assuming the Markov property holds for the world):
* __Learning rate decay__: The learning rate $\alpha_{t}$ must satisfy $\sum_{t=0}^\infty \alpha_t(s, a) = \infty$ and $\sum_{t=0}^\infty \alpha_t^2(s, a) < \infty$ for all state-action pairs, ensuring sufficient initial updates while stabilizing over time. Thus, $\alpha_t \gets \beta\alpha_{t}$ where $\beta<1$ are common choices.
* __Infinite exploration__: All state-action pairs _can_ be visited infinitely often. This condition is true for $\epsilon$-greedy policies with persistent exploration, i.e., $\epsilon_{t} > 0\,\,\forall{t}$.

See the [Q-learning lecture notes from L8c for more details](https://htmlview.glitch.me/?https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-8/L8c/CHEME-5820-L8c-QLearning-S2025.html).

## Deep Q-Learning (DQN)
Deep Q-learning is a variant of Q-learning that deep neural networks to approximate the Q-value function.

<img src="figs/Q-Learning-vs-Deep-Q-Learning.ppm.png" alt="Q-Learning vs Deep Q-Learning" width="600"/>

This approach allows for the handling of high-dimensional state spaces, such as images or continuous states, where traditional Q-learning would be infeasible due to the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality).
* _How does approach word_? In this approach, the Q-value function $Q(s,a)$ is represented as a neural network, which takes the state $s$ as input and outputs the Q-values for all possible actions. The neural network is trained using the same Q-learning update rule, but with mini-batches of experiences sampled from a replay buffer to stabilize training.
* _Games_? This approach was made famous by [the DeepMind team in 2015](https://www.nature.com/articles/nature14236), where they used DQN to play Atari games directly from pixels. This approach achieved human-level performance on other games, e.g., DQN was used as part of the policy network [pre-training phase for AlphaGo](https://doi.org/10.1038/nature16961), the first AI to defeat a professional human Go player, marking a milestone in AI. 
* _Other applications?_ DQN has been used in other applications such as operations management, e.g., a [DQN-based system was deployed in Google’s data centers to optimize cooling, achieving a reported 30% reduction in energy consumption for cooling systems](https://deepmind.google/discover/blog/deepmind-ai-reduces-google-data-centre-cooling-bill-by-40/) or [traffic signal control in smart cities](https://dl.acm.org/doi/10.1145/3219819.3220096), where DQN was used to optimize traffic light timings in real-time, leading to reduced congestion and improved traffic flow.


### Theory of DQN
A deep Q-learning agent learns a policy $\pi$ that maximizes the expected cumulative reward $R_t$ over time. Suppose the agent is task with making decisons over 

For each epsiode, we sample for $t = 1,2,\ldots,T$: 

1. __Interaction with the environment__: At each time step $t$, the agent observes the current state $s_t$, selects an action $a_t$ (typically using an $\epsilon$-greedy policy based on the _Q-network_), and receives a reward $r_t$ and the next state $s_{t+1}$ from the environment.
2. __Experience replay__: Each transition tuple $(s_t, a_t, r_t, s_{t+1})$ is stored in a **replay buffer** (a finite-sized memory that we'll use for training). Instead of training on consecutive samples, the agent **samples random mini-batches** from this buffer. 
3. __Main Q-Network (Function Approximator)__: The core of DQN is a deep neural network $Q_{\theta}(s)$ with (trainable) parameters $\theta$, which learns to approximate the optimal action-value function. The network takes a state as input and outputs Q-values for all possible actions.
4. __Target Q-Network__: To stabilize training, DQN uses a **target network** $Q^{\prime}_{\theta^{-}}(s)$, which is a delayed copy of the main Q-network. The target network’s parameters $\theta^-$ are updated periodically (e.g., every $N$ steps) by copying the weights from the main Q-network.


#### DQN Algorithm

__Initialize__ the parameters of the main Q-network $Q_{\theta}(s)$ and the target Q-network $Q^{\prime}_{\theta^{-}}(s)$ to random values. Initialize a (potentially inifinite) replay buffer $\mathcal{B}$. Set the hyperparameters: the learning rate $\alpha$, the discount factor $\gamma$, the exploration rate $\epsilon_{t}$, the minimum number of experiences in the replay buffer $B$, and the parameter update count $\mathcal{C}$.
- For each episode, intialize the state to $s_0$ and:
   - For each time step $t=1,\ldots,T$:
        1. Role a random number $p\in[0,1]$. If $p\leq\epsilon_{t}$, choose a random (uniform) action $a_{t}\in\mathcal{A}$. Otherwise, choose a greedy action $a_{t} = \text{arg}\max_{a\in\mathcal{A}}{Q_{\theta}(s_{t})}$.
        2. Execute action $a_{t}$, observe the reward $r_{t}$ from the _world_ and transition to the next state $s_{t+1}$. 
        3. Store the transition (experience) $\mathcal{e}=(s_t, a_t, r_t, s_{t+1})$ in the replay buffer: $\mathcal{e}\rightarrow\mathcal{B}$. 
        5. If the replay buffer $\mathcal{B}$ has a _minium number of elements_: sample a mini-batch of experiances $(s_i, a_i, r_i, s_{i+1})$ from the replay buffer.  The agent randomly samples a mini-batch of $B$ transitions from the replay buffer:  $(s_j, a_j, r_j, s_{j+1}),\, j = 1, 2, \dots, B$. Each tuple represents a state-action-reward-next state experience example collected during environment interaction.
        6. Compute the _target Q-value_ for each transition in the mini-batch using the _target Q-network_: $y_i = r_i + \gamma \cdot \max_{a^{\prime}\in\mathcal{A}}Q^{\prime}_{\theta^{-}}(s_{i+1})$ for $i=1,2,\ldots,B$.
        7. Compute the _mean squared loss_ function over the $B$ experiances collected in the mini-batch: $L(\theta) = \frac{1}{B}\sum_{i=1}^{B}\left(y_i - Q_{\theta}(s_i)\right)^2$.
        8. Perform a _single_ gradient descent step to minimize the loss function $L(\theta)$ with respect to the parameters $\theta$ of the main Q-network $Q_{\theta}(s)$: $\theta \leftarrow \theta - \alpha \nabla_{\theta}L(\theta)$, where $\alpha$ is the learning rate. 
            - _Why only a single step_? Each mini-batch is just a _small sample of the environment’s dynamics._ The goal of DQN is _online learning_: the network parameters are continuously updated as new experiences come in. If we force training to convergence on each mini-batch risks _overfitting to that mini-batch_.
        10. Update the state $s_t \leftarrow s_{t+1}$.
        9. Every $C$ steps, update the target Q-network parameters: $\theta^{-} \leftarrow \theta$.
        
    - End For
- End For

## Practical details
There are several practical details that are important to the success of DQN, largely focused around the training process and the management of the replay buffer. These details are critical for the stability and performance of DQN, especially in complex environments.

### Replay buffer management
The replay buffer is a key component of DQN, allowing the agent to learn from past experiences. The buffer stores transitions (state, action, reward, next state) and samples mini-batches for training. Thus, one obvious question is: _How do we manage the replay buffer_?

* In DQN, the **replay buffer has a fixed maximum size** (often denoted as $M$, e.g., 100,000 or 1,000,000, etc). This is done for both memory efficiency and to ensure that the agent focuses on more recent, relevant experiences.
* When the buffer reaches its maximum size, the oldest experiences are discarded to make room for new ones, e.g., in a first-in, first-out manner. This ensures that the agent learns from a diverse set of experiences and avoids overfitting to outdated information.
* This approach ensures that the replay buffer contains a mix of old and new experiences, prioritizes more recent experiences over time,and stays within a fixed memory footprint.

### Mini-batch construction
Each mini-batch is sampled randomly from the current contents of the buffer. Older experiences are still used for training as long as they remain in the buffer. However, once an experience is overwritten (evicted), it no longer contributes to future mini-batches. Thus, the mini-batch is a random sample of old and new experiences from the replay buffer. 

* __Vanilla DQN__: The basic replay buffer in DQN is typically implemented using a [circular buffer (ring buffer)](https://en.wikipedia.org/wiki/Circular_buffer#:~:text=In%20computer%20science%2C%20a%20circular,easily%20to%20buffering%20data%20streams.), which is conceptually similar to a fixed-size queue. This data structure has with constant-time insert and sample operations, making it efficient for storing and sampling experiences.
* __Prioritized Replay DQN__: When using **prioritized experience replay** (where experiences are sampled with different probabilities based on their _importance_), a more advanced data structure is needed, e.g., a [binary heap](https://en.wikipedia.org/wiki/Binary_heap#:~:text=A%20binary%20heap%20is%20a,data%20structure%20for%20implementing%20heapsort.) or a [sum tree](https://en.wikipedia.org/wiki/Fenwick_tree). This allows for efficient sampling of experiences based on their priority.

### Training frequency
In practice, DQN _does not_ train the network immediately at every step from the start. Instead, training begins after the replay buffer has accumulated a minimum number of transitions (often called the _warm-up period_). This ensures that each mini-batch used for training contains diverse and meaningful experiences, which stabilizes learning.

* __Warm-Up Phase__: The agent interacts with the environment and **stores transitions** in the replay buffer **without updating the Q-network** until the buffer reaches a threshold size (e.g., 1,000 or 10,000 transitions, depending on the problem).
* __Training Phase__: Once the replay buffer exceeds the threshold, training begins. After that point: The network can be updated at every environment step. Alternatively, it can be updated at a fixed interval (e.g., every 4 environment steps) to save computation.
* Thus, a typical pattern is: (i) Fill replay buffer to a minimum size, (ii) Start training, and (iii) Continue to fill the buffer with new experiences while training.

## Lab
In `L16b`, we will implement a simple Deep Q-Learning (DQN) agent. The goal is to train the agent to play the continuous valued game, where the agent will use a neural network to approximate the Q-values for each action given a state.

# Today?
That's a wrap! What are some of the interesting things we discussed today?